# pytorch 기반 alexnet 구현

In [ ]:
im port torch
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(),
            nn.LocalResponseNorm(5),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.LocalResponseNorm(5),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes) #파이토치의 규칙 때문에 Linear로 마무리, Cross Entropy Loss에서 계산
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1) # == flatten()
        x = self.classifier(x)
        return x

In [ ]:
model = AlexNet(num_classes=10)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=

In [ ]:
# AlexNet 모델을 간소화해봅시다
## resize없이 (28, 28, 3)으로 받아봅시다

## ** activation 은 모두 relu로 유지
## normalizing 유지
## pooling은 (2, 2) overlapping x

## layer 1 : 32개, (3, 3), strides=1
## layer 2 : 64개, (3, 3), strides=1
## layer 3 : 128개, (3, 3), strides=1

## layer 4, 5 : 삭제

## fully connected layer 1, 2 : node 원하는대로 주기


In [ ]:
def train():
   # 디바이스 설정
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   # 모델 생성
   model = AlexNet(num_classes=10).to(device)

   # Loss function과 optimizer 정의
   criterion = nn.CrossEntropyLoss()
   optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

   # 데이터 로더 예시 (실제로는 여기에 데이터셋 로딩 코드 필요)
   # train_loader = ...

   # 학습 루프
   num_epochs = 90
   for epoch in range(num_epochs):
       model.train()
       running_loss = 0.0

       for inputs, labels in train_loader:
           inputs, labels = inputs.to(device), labels.to(device)

           # gradient 초기화
           optimizer.zero_grad()

           # forward + backward + optimize
           outputs = model(inputs)
           loss = criterion(outputs, labels)
           loss.backward()
           optimizer.step()

           running_loss += loss.item()

       print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}')